In [1]:
# import and install packages

!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm
!pip install google-colab

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from google.colab import files
from google.colab import drive
import json
from PIL import Image, ImageDraw
from IPython.core.ultratb import AutoFormattedTB
from PIL import Image
import glob
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

# assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00


Load Images

In [2]:
drive.mount('/content/drive')
path_coco = "/content/damage_coco"
path_robo = "/content/damage_robo"

if not os.path.exists(path_coco):
    os.makedirs(path_coco)
if not os.path.exists(path_robo):
    os.makedirs(path_robo)

Mounted at /content/drive


In [3]:
!unzip -q /content/drive/MyDrive/College/3.\ Junior/Winter\ Semester/CS\ 474/Final\ DL/Car_Data/damage_coco.zip -d /content/damage_coco
!unzip -q /content/drive/MyDrive/College/3.\ Junior/Winter\ Semester/CS\ 474/Final\ DL/Car_Data/damage_roboflow.zip -d /content/damage_robo
# !unzip  -q /content/drive/MyDrive/DL_Project/damage_coco.zip -d /content/damage_coco
# !unzip -q /content/drive/MyDrive/DL_Project/damage_roboflow.zip -d /content/damage_robo

In [4]:
def resize(filename):
  root_folder = "/content/damage_coco/train"
  for subdir, dirs, files in os.walk(root_folder):
      for file in files:
          # get the file extension and check if it's an image
          ext = os.path.splitext(file)[-1].lower()
          
          if ext in ['.jpg', '.jpeg', '.png', '.bmp']:
              # get the full path of the image file
              file_path = os.path.join(subdir, file)
              
              # open the image and resize it to a smaller size
              img = Image.open(file_path)
              os.remove(file_path)
              img = img.resize((256, 256), resample=Image.LANCZOS)
              
              # save the compressed image in the same folder with a new name
              new_file_name = os.path.splitext(file)[0] + ext
              img.save(os.path.join(subdir, new_file_name), quality=70)


In [5]:
# resize damage_coco training images
resize("/content/damage_coco/train")

# resize damage_coco test images
resize("/content/damage_coco/test")

# resize damage_roboflow training images
resize("/content/damage_roboflow/train")


In [6]:
# !rmdir /content/damage_coco/val/.ipynb_checkpoints/

In [7]:
# make a new file path for the training images for the data loader
path_coco_train = "/content/damage_coco/train/0"
path_coco_test = "/content/damage_coco/test/0"

if not os.path.exists(path_coco_train):
    os.makedirs(path_coco_train)
if not os.path.exists(path_coco_test):
    os.makedirs(path_coco_test)


path_robo_train = "/content/damage_robo/train/0"
path_robo_test = "/content/damage_robo/test/0"

if not os.path.exists(path_robo_train):
    os.makedirs(path_robo_train)
if not os.path.exists(path_robo_test):
    os.makedirs(path_robo_test)

# move training images:

In [8]:
!mv /content/damage_coco/train/*.jpg /content/damage_coco/train/0
!mv /content/damage_coco/test/*.jpg /content/damage_coco/test/0

!mv /content/damage_robo/train/test_set_*.jpg /content/damage_robo/test/0
!mv /content/damage_robo/train/*.jpg /content/damage_robo/train/0

!rm /content/damage_coco/img/*.jpg
!rmdir/content/damage_coco/img

/bin/bash: rmdir/content/damage_coco/img: No such file or directory


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# STANFORD ANNOTATION COCO DATA
ratio = 4
size = 256
width, height = size, size
test_ids = [11, 12, 28, 45, 60, 66, 67, 72]

# make a new file path for the training masks for the data loader
coco_mask_train = "/content/damage_coco/masks_train/0"
coco_mask_test = "/content/damage_coco/masks_test/0"
if not os.path.exists(coco_mask_train):
    os.makedirs(coco_mask_train)
if not os.path.exists(coco_mask_test):
    os.makedirs(coco_mask_test)


# Load the JSON data
with open('/content/damage_coco/train/COCO_mul_train_annos.json') as f:
    data = json.load(f)

# Create an empty mask
mask = np.zeros((height, width), dtype=np.uint8)
last_image = 0
damage_count = 0

for annotation in data['annotations']:
  # print(last_image)
  image_id = annotation['image_id']
  if last_image == image_id:
    damage_count += 1

  else:
    # save the previous mask
    if image_id in test_ids:
      folder = "masks_test"
    else:
      folder = "masks_train"
    name = '/content/damage_coco/' + folder + '/0/mask'+ \
            str(image_id)+'_'+str(damage_count)+'.png'
    # print(name)

    # Convert the array to an image and save it
    img = Image.fromarray(mask*255)
    img.save(name)
    # img.show()
    if last_image == 13:
      break

    # Create an empty mask
    mask = np.zeros((height, width), dtype=np.uint8)
    last_image = annotation['image_id']
    damage_count = 0

  # get damage labels
  box = annotation['bbox']
  x, y, dx, dy = [b // ratio for b in box]
  # print("box:", box)
  # print("xyd:", x, y, dx, dy)
  mask[x:x+dx+1, y:y+dy+1] +=1

# STANFORD ANNOTATION COCO DATA

In [11]:
# ROBOFLOW ANNOTATION DATA
ratio = 2
size = 256
width, height = size, size

# make a new file path for the training masks for the data loader
robo_mask_train = "/content/damage_robo/masks_train/0"
robot_mask_test = "/content/damage_robo/masks_test/0"
if not os.path.exists(robo_mask_train):
    os.makedirs(robo_mask_train)
if not os.path.exists(robot_mask_test):
    os.makedirs(robot_mask_test)


# Load the JSON data
with open('/content/damage_robo/train/_annotations.coco.json') as f:
    data = json.load(f)

image_id = 0

for annotation in data['annotations']:
  # Create an empty mask
  mask = np.zeros((height, width), dtype=np.uint8)
  image_id = annotation['image_id']

  # get damage labels
  box = annotation['bbox']
  x, y, dx, dy = [int(b // ratio) for b in box]
  mask[x:x+dx+1, y:y+dy+1] +=1

  # determine if the data is for testing or training
  meta_image = next((img for img in data['images'] if img['id'] == image_id), None)
  if meta_image and meta_image['file_name'].startswith('test'):
      folder_name = 'masks_test'
  else:
      folder_name = 'masks_train'

  # save the mask
  name = '/content/damage_robo/' + folder_name + '/0/mask'+ \
          str(image_id) + '.png'

  # Convert the array to an image and save it
  img = Image.fromarray(mask*255)
  img.save(name)
  
# ROBOFLOW ANNOTATION DATA

In [12]:
class DamagedDataset(Dataset):
  def __init__(self, root1, root2=None, download=True, size=256, train=True):
    postfix = 'train' if train else 'test'
    label_post = 'masks_train' if train else 'masks_test'
    print(os.path.join(root1, postfix))

    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root1, postfix), 
                                                           transform = transforms.Compose([
                                                               transforms.Resize(size),
                                                               transforms.ToTensor()
                                                               ]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root1, label_post), 
                                                         transform = transforms.Compose([
                                                             transforms.Resize(size),
                                                             transforms.ToTensor()
                                                             ]))
    
    if root2 is not None:
      self.dataset_folder2 = torchvision.datasets.ImageFolder(os.path.join(root2, postfix), 
                                                            transform = transforms.Compose([
                                                                transforms.Resize(size),
                                                                transforms.ToTensor()
                                                                ]))
      self.label_folder2 = torchvision.datasets.ImageFolder(os.path.join(root2, label_post), 
                                                          transform = transforms.Compose([
                                                              transforms.Resize(size),
                                                              transforms.ToTensor()
                                                              ]))
      
      # Combine the two dataset and label folders
      self.dataset_folder = torch.utils.data.ConcatDataset([self.dataset_folder, self.dataset_folder2])
      self.label_folder = torch.utils.data.ConcatDataset([self.label_folder, self.label_folder2])


  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0], label[0][0]
  
  
  def __len__(self):
    return len(self.dataset_folder)

end data loading
.---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Neural Net

In [13]:
class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, up_sample=False):
    super(ConvBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, (3,3), padding=(1,1))
    self.conv2 = nn.Conv2d(out_channels, out_channels, (3,3), padding=(1,1))
    self.up_sample = up_sample
    if up_sample:    
      # Up sample with up-conv 2x2, doubling along each dimension
      self.up = nn.ConvTranspose2d(out_channels, out_channels//2, 2, stride=2)
    # We do not want to down sample in the block, since we need the non-down-sampled output for skip connections

  def forward(self, input):
    x = F.relu(self.conv1(input))
    x = F.relu(self.conv2(x))
    if self.up_sample:
      x = self.up(x)
    return x

class DamageDetection(nn.Module):
  def __init__(self):
    super(DamageDetection, self).__init__()
    # Downsample convolution blocks
    self.dblock1 = ConvBlock(3, 32)
    self.dblock2 = ConvBlock(32, 64)
    self.dblock3 = ConvBlock(64, 128)
    self.dblock4 = ConvBlock(128, 256)
    # Upsample convolution blocks
    self.ublock1 = ConvBlock(256, 512, True)
    # The next 4 blocks have doubled input channels due to concatenation of skip connections
    self.ublock2 = ConvBlock(512, 256, True)
    self.ublock3 = ConvBlock(256, 128, True)
    self.ublock4 = ConvBlock(128, 64, True)
    # Output "block"
    self.convf1 = nn.Conv2d(64, 32, (3,3), padding=(1,1))
    self.convf2 = nn.Conv2d(32, 32, (3,3), padding=(1,1))
    self.convf3 = nn.Conv2d(32, 2, (1,1), padding=(0,0))

    self.down = nn.MaxPool2d(2)
 
  def forward(self, input):
    # Save the last feature maps on each level! Pass an instance forward, but keep the variable referencing the same feature map for concatenation
    l1 = self.dblock1(input)
    l2 = self.dblock2(self.down(l1))
    l3 = self.dblock3(self.down(l2))
    l4 = self.dblock4(self.down(l3))
    # Concatinate l1 - l4 on inputs across the U in reverse order, matching sizes
    u = self.ublock1(self.down(l4))
    u = self.ublock2(torch.cat((l4, u), dim=1))
    u = self.ublock3(torch.cat((l3, u), dim=1))
    u = self.ublock4(torch.cat((l2, u), dim=1))
    out = F.relu(self.convf1(torch.cat((l1, u), dim=1)))
    out = F.relu(self.convf2(out))
    out = self.convf3(out)
    return out

Initialize Classifier

Create datasets, dataloaders and neural network

In [14]:
# TODO load masks so that they can load properly with the DamagedDataset class






# Initialize Datasets
path_coco = '/content/damage_coco'
path_robo = '/content/damage_robo'
train_dataset = DamagedDataset(path_coco, path_robo, train = True)
val_dataset = DamagedDataset(path_coco, path_robo, train = False)

# Initialize Model
model = DamageDetection()
model = model.cuda()
 # Initialize Objective and Optimizer and other parameters
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/content/damage_coco/train
/content/damage_coco/test


In [15]:
losses = []
accuracies = []
validations = []
val_accs = []
pred_img = []

Train

In [16]:
def scope():
  try:
    #your code for calling dataset and dataloader
    train_loader = DataLoader(train_dataset, 
                                  batch_size = 6,
                                  num_workers = 2, 
                                  pin_memory = True,
                                  shuffle = True)
    val_loader = DataLoader(val_dataset, 
                            batch_size = 6,
                            num_workers = 2,
                            pin_memory = True,
                            shuffle = True)
    gc.collect()
    print(torch.cuda.memory_allocated() / 1e9)
    
    val_acc = 0

    # Run your training and validation loop and collect stats
    for epoch in range(2):

      loop = tqdm(total=len(train_loader), position=0, leave=False)

      for batch, (x, y_truth) in enumerate(train_loader):
        x, y_truth = x.cuda(), y_truth.cuda()
        
        # Call your model, figure out loss and accuracy
        optimizer.zero_grad()
        y_hat = model(x)

        loss = objective(y_hat, y_truth.long())
        loss.backward()
        losses.append(loss.item())

        accuracy = (torch.softmax(y_hat, 1).argmax(1) == y_truth).float().mean()
        accuracies.append(accuracy.cpu())
        loop.set_description('epoch{}, loss:{:.4f}, accuracy:{:.3f} val_acc:{:.3f}'.format(epoch, loss.item(), accuracy, val_acc))
        loop.update(1)

        optimizer.step()

        if batch % 20 == 0:
          with torch.no_grad():
            val = np.mean([objective(model(x.cuda()), y.cuda().long()).item() for x, y in val_loader])
            val_acc = np.mean([(model(x.cuda()).argmax(1) == y.cuda().long()).float().mean().item() for x, y in val_loader])
            validations.append((len(losses), val))
            val_accs.append((len(accuracies), val_acc))

      pred_img.append(model(val_dataset[-1][0].unsqueeze(0).cuda()))

      loop.close()

  except:
    __ITB__()
    
scope()

0.031044608


epoch0, loss:0.6332, accuracy:0.915 val_acc:0.000:   0%|          | 1/264 [00:02<08:59,  2.05s/it]

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<ipython-input-16-af93d8120b1f> in <listcomp>(.0=<torch.utils.data.dataloader._MultiProcessingDataLoaderIter object>)
     42         if batch % 20 == 0:
     43           with torch.no_grad():
---> 44             val = np.mean([objective(model(x.cuda()), y.cuda().long()).item() for x, y in val_loader])
        global val = undefined
        global np.mean = <function mean at 0x7f82c819e280>
        global objective = CrossEntropyLoss()
        global model = DamageDetection(
  (dblock1): ConvBlock(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (dblock2): ConvBlock(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


Show Accuracy

In [17]:
a, b = zip(*validations)

plt.plot(losses, label="Train")
plt.plot(a, b, label="Validation")
plt.xlabel("Steps")
plt.ylabel("Error")
plt.ylim(0, 1)
plt.title("Model Error")
plt.legend()
plt.show()

ValueError: ignored

In [ ]:
a, b = zip(*val_accs)
print(f"Final Accuracy: {b[-1]}")

plt.plot(accuracies, label="Train")
plt.plot(a, b, label="Validation")
plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Model Accuracy")
plt.legend()
plt.show()

Testing

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('/content/damage_coco/test/72.jpg')
plt.imshow(img)
plt.title("Original Image")
print(img.shape)

In [ ]:
for i, p in enumerate(pred_img):
  q = p.squeeze(0).argmax(0).cpu().detach().numpy()
  plt.imshow(q, cmap="gray")
  plt.title(f"Classification after Epoch {i}")
  print(np.sum(q))

In [ ]:
print(len(train_dataset))

In [ ]:
for i in range(len(train_dataset)):
  q = model(train_dataset[i][0].unsqueeze(0).cuda()).squeeze(0).argmax(0).cpu().detach().numpy()
  if np.sum(q) != 0:
    print(i, np.sum(q))